In [ ]:
# processes PM2.5 data

In [1]:
import os
import sys
import math
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from pathlib import Path
import glob
import numpy as np
import rasterio.mask
import rasterio
import fiona
from fiona.crs import from_epsg
from pathlib import Path

In [3]:
aoi_folder = Path('data/AOI')
city_list = pd.read_csv('centroids.csv').city
output_folder = Path('data')

In [6]:
def clipdata_air(city, year):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    file = city / aoi_folder / (city_lower + '_AOI.shp')
    
    with fiona.open(file, "r") as shapefile:
        features = [feature["geometry"] for feature in shapefile]
        
        # Raw data folder. Change file path as needed
        input_raster = "F:/World Bank/City Scan/data/Global Annual PM2.5 Grids 1998-2019/sdei-global-annual-gwr-pm2-5-modis-misr-seawifs-aod-v4-gl-03-" + str(year) + ".tif"
        with rasterio.open(input_raster) as src:
            # shapely presumes all operations on two or more features exist in the same Cartesian plane.
            out_image, out_transform = rasterio.mask.mask(
                src, features, crop=True)
            out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        output_file = city_lower + '_air_quality_' + str(year) + '.tif'
        with rasterio.open(Path(city) / output_folder / output_file, "w", **out_meta) as dest:
            dest.write(out_image)

In [7]:
for city in city_list:
    for year in range(1998, 2020):
        clipdata_air(city, year)